# MetaKG `x-bte` Parser Inspection 

In [43]:
import warnings
import yaml
import json
import pprint
 
# Import the databases
from controller.smartapi import SmartAPI
from controller.metakg import MetaKG
from model import ConsolidatedMetaKGDoc
from utils.metakg import parser

warnings.filterwarnings('ignore')

### Setup Parameters

In [44]:
parser = parser.MetaKGParser()

In [45]:
# # set index name for metakg traversal
# index = ConsolidatedMetaKGDoc.Index.name

In [46]:
def compare_keys(actual, expected, path=""):
    missing_keys = []
    extra_keys = []

    # Check for missing keys
    for key, value in expected.items():
        if key not in actual:
            missing_keys.append(f"{path}.{key}" if path else key)
        elif isinstance(value, dict) and isinstance(actual.get(key), dict):
            # Special handling for 'operation_name_pattern' keys
            if key == "operation_name_pattern":
                for actual_key in actual.keys():
                    nested_missing, nested_extra = compare_keys(actual[actual_key], value, f"{path}.{actual_key}" if path else actual_key)
                    missing_keys.extend(nested_missing)
                    extra_keys.extend(nested_extra)
            else:
                nested_missing, nested_extra = compare_keys(actual[key], value, f"{path}.{key}" if path else key)
                missing_keys.extend(nested_missing)
                extra_keys.extend(nested_extra)

    # Check for extra keys
    for key in actual.keys():
        if key not in expected:
            if "operation_name_pattern" in expected:
                # If there's an 'operation_name_pattern', consider the key valid
                continue
            extra_keys.append(f"{path}.{key}" if path else key)

    return missing_keys, extra_keys


In [47]:
def flatten_actual_keys(d, parent_key=''):
    """
    Recursively flattens the keys of a nested dictionary into a set of key paths.
    Removes the 'api.' prefix from each key.
    """
    items = set()
    for k, v in d.items():
        new_key = f"{parent_key}.{k}" if parent_key else k
        if isinstance(v, dict):  # If the value is a dictionary
            items.update(flatten_actual_keys(v, new_key))
        else:
            items.add(new_key)
    
    # Remove 'api.' from all keys
    cleaned_items = {key.replace('api.', '') for key in items}
    return cleaned_items


def flatten_actual_keys(d, parent_key=''):
    """
    Recursively flattens the keys of a nested dictionary into a set of key paths.
    """
    items = set()
    for k, v in d.items():
        new_key = f"{parent_key}.{k}" if parent_key else k
        if isinstance(v, dict):  # If the value is a dictionary
            items.update(flatten_actual_keys(v, new_key))
        else:
            items.add(new_key)
    return items


In [48]:
expected_keys = {
    "components": {
        "x-bte-kgs-operations": {
            "operation_name_pattern": {
                "supportBatch": None,
                "useTemplating": None,
                "inputs": {
                    "id": None,
                    "semantic": None
                },
                "requestBody": {
                    "body": None
                },
                "requestBodyType": None,
                "outputs": {
                    "id": None,
                    "semantic": None
                },
                "parameters": {
                    "fields": None,
                    "size": None
                },
                "predicate": None,
                "source": None,
                "qualifiers": None,
                "knowledge_level": None,
                "agent_type": None,
                "testExamples": {
                    "qInput": None,
                    "oneOutput": None
                },
                "response_mapping": None
            }
        },
        "x-bte-response-mapping": {
            "source_url": None,
            "edge-attributes": None,
            "trapi_sources": None,
            "ref_input": None,
            "ref_output": None,
            "input_name": None,
            "output_name": None
        }
    }
}

# # Flatten the expected_keys dictionary into a list of key names
# expected_key_names = list(flatten_keys(expected_keys))
# expected_key_names


In [49]:

for smartapi in SmartAPI.get_all(10):
    # get and load document
    print()
    ct+=1
    raw_data = smartapi.raw.decode('utf-8')
    source_data = yaml.safe_load(raw_data)
    # print(f"Document: {ct}")
    # Extract MetaKG Edges
    metakg = parser.get_non_TRAPI_metadatas(data=source_data)
    if metakg:
        print(f"MetaKG doc: {ct}")
        # print("Top-level keys in the document:", source_data.keys())
        print("-"*50)
        print("\nDocuments Extracted MetaKG Edges:")

        # Verify `x-bte` Extraction
        for edge in metakg:
            print(edge)
            # print("\nEdge API BTE Data:", json.dumps(edge.get("api", {}).get("bte", {}), indent=2))
            # Flatten the keys in the edge
            actual_key_names = list(flatten_actual_keys(edge))
            print("Document keys: ",actual_key_names)

        #     print(edge['api']['bte'].keys())
        #     print(actual_key_names)
        #     # Compare the actual keys with the expected keys
        #     missing_keys = list(set(expected_key_names) - set(actual_key_names))
        #     extra_keys = list(set(actual_key_names) - set(expected_key_names))

        # #     print("\nExpected Keys:", expected_key_names)
        #     print("Found Keys:", actual_key_names)
        #     if missing_keys:
        #         print("Missing Keys:", missing_keys)
        #     if extra_keys:
        #         print("Extra Keys:", extra_keys)
        # print("-"*50)

ConnectionError: Connection error caused by: ConnectionError(Connection error caused by: NewConnectionError(<urllib3.connection.HTTPConnection object at 0x105ea1e40>: Failed to establish a new connection: [Errno 61] Connection refused))

---

Report Target Points:  
Collect missing keys from documents  
Collect extra(unexpected) keys from documents  
Gather a collective analysis and individual analysis   


---

---